## 0. Import necessary libraries

In [1]:
import numpy as np
import pandas as pd

## 1. Get data from a previosly made function

In [33]:
from preparation import prepare_data

PATH = "C:/Users/San/Documents/CS projects/yt_activity_analysis/data/Takeout/YouTube and YouTube Music/history/watch-history.json"
df = prepare_data(PATH)
df.head(5)

,video_title,channel_name,time
0,Building Search Word Database - Data Analysis of Google Tracking Takeout p.1,sentdex,2022-07-04 06:25:34.777000+00:00
1,How to Use Google Takeout in 2022,Richard Byrne,2022-07-04 06:22:34.867000+00:00
2,How to Download Your Google Data || Google Takeout || Download your data in just one click,Data Guru (Data Science),2022-07-04 06:22:22.071000+00:00
3,"Марш Українського війська ""Зродились ми великої години""",Ser General,2022-07-04 05:54:41.842000+00:00
4,Чорне і біле,ZOZULYA - Topic,2022-07-04 05:51:05.598000+00:00


## 2. Time to explore! 